# Exploring convolution

First, include some libraries

In [ ]:
# Library imports

import math
import numpy as np

#
# Import tensor class
#
from fibertree import Fiber, Tensor, TensorImage

#
# Import display classes/utilities
#
from IPython.display import display # to display images

def displayTensor(t):
    display(TensorImage(t).im)

#
# Matplolib classes (not currently used)
#
from matplotlib.pyplot import imshow

#
# Import rc to configure animation for HTML5
#
from matplotlib import rc
rc('animation', html='html5')

#
# Helper for data directory
#
import os

data_dir = "../data"

def datafileName(filename):
    return os.path.join(data_dir, filename)



## Convolution Inputs


In [ ]:
w = Tensor(os.path.join(data_dir, "conv-weights-a.yaml"))
i = Tensor(os.path.join(data_dir, "conv-activations-a.yaml"))

displayTensor(w)
displayTensor(i)

## Weight Stationary

In [ ]:
o = Tensor(rank_ids=["Q"])

displayTensor(o)

w_r = w.root()
i_h = i.root()
o_q = o.root()

W = w_r.maxCoord() + 1
I = i_h.maxCoord() + 1
Q = I - W + 1

print("Convolution")

for r, (w_val) in w_r:
    print(f"Processing weight: ({r}, ({w_val}))")
    for q, (o_q_ref, i_val) in o_q << i_h.project(lambda h: h-r, (0, Q)):
        print(f"  Processing output ({q}, ({o_q_ref}, {i_val})")
        o_q_ref += w_val * i_val

displayTensor(o)

## Input Stationary

In [ ]:
o = Tensor(rank_ids=["Q"])

displayTensor(o)

w_r = w.root()
i_h = i.root()
o_q = o.root()

W = w_r.maxCoord() + 1
I = i_h.maxCoord() + 1
Q = I - W + 1

print("Convolution")

for h, (i_val) in i_h:
    print(f"Processing input: ({h}, ({i_val}))")
    for q, (o_q_ref, w_val) in o_q << w_r.project(lambda r: h-r, (0, Q)):
        print(f"  Processing output ({q}, ({o_q_ref}, {w_val})")
        o_q_ref += w_val * i_val


displayTensor(o)


## Output Stationary

In [ ]:
o = Tensor(rank_ids=["Q"])

displayTensor(o)

w_r = w.root()
i_h = i.root()
o_q = o.root()

W = w_r.maxCoord() + 1
I = i_h.maxCoord() + 1
Q = I - W + 1

print("Convolution")

output_shape = Fiber(coords=range(Q), initial=1)

displayTensor(output_shape)

for q, (o_q_ref, _) in o_q << output_shape:
    print(f"Processing output: ({q}, ({o_q_ref}))")
    for r, (w_val, i_val) in w_r.project(lambda r: q+r) & i_h:
        print(f"  Processing weights and activations ({r}, ({w_val}, {i_val})")
        o_q_ref += w_val * i_val


displayTensor(o)


## Output Stationary - Two pass

In [ ]:
o = Tensor(rank_ids=["Q"])

displayTensor(o)

w_r = w.root()
i_h = i.root()
o_q = o.root()

W = w_r.maxCoord() + 1
I = i_h.maxCoord() + 1
Q = I - W + 1

print("Convolution")

pass1_count = 0

for r, (_) in w_r:
    print(f"Processing weight: ({r}, (_))")
    for q, (o_q_ref, _) in o_q << i_h.project(lambda h: h-r, (0, Q)):
        print(f"  Calculating output ({q}, ({o_q_ref}, _)")
        pass1_count += 1

print("Pass1 count: %s" % pass1_count)
displayTensor(o_q)

for q, (o_q_ref) in o_q:
    print(f"Processing output: ({q}, ({o_q_ref}))")
    for r, (w_val, i_val) in w_r.project(lambda r: q+r) & i_h:
        print(f"  Processing weights and activations ({r}, ({w_val}, {i_val})")
        o_q_ref += w_val * i_val


displayTensor(o)


## Output Stationary - Two pass - Optimized

In [ ]:
o = Tensor(rank_ids=["Q"])

displayTensor(o)

w_r = w.root()
i_h = i.root()
o_q = o.root()

W = w_r.maxCoord() + 1
I = i_h.maxCoord() + 1
Q = I - W + 1

print("Convolution")

pass1_count = 0

for r, (_) in w_r:
    print(f"Processing weight: ({r}, (_))")
    for q, (o_q_ref, _) in o_q << i_h.project(lambda h: h-r, (0, Q)) - o_q:
        print(f"  Calculating output ({q}, ({o_q_ref}, _)")
        pass1_count += 1

print("Pass1 count: %s" % pass1_count)

for q, (o_q_ref) in o_q:
    print(f"Processing output: ({q}, ({o_q_ref}))")
    for r, (w_val, i_val) in w_r.project(lambda r: q+r) & i_h:
        print(f"  Processing weights and activations ({r}, ({w_val}, {i_val})")
        o_q_ref += w_val * i_val


displayTensor(o)


## Testing area

For running alternative algorithms